In [ ]:
import feedparser
import dateutil.parser
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import mongo_utils

In [ ]:
dbname = mongo_utils.get_database()

In [ ]:
urls = ["http://www.focus-news.net/RSS/channel_0.xml",
    ]
feeds = [feedparser.parse(url)['entries'] for url in urls]
for url in urls:
    print(url)
#feeds[1][0].keys()
feed = [item for feed in feeds for item in feed]

In [ ]:
#print(feed)

In [ ]:
for newslink in feed:
  try:
    #print(newslink["id"])
    nlink=newslink["link"]
    #print(nlink)
    date_posted=newslink["published"]
    news_title=newslink["title"]
    text=newslink["summary"]
    page = requests.get(nlink)
    data = page.text
    soup = BeautifulSoup(data, 'html.parser')
    #text = soup.find_all('p')
    text = soup.find('div', class_='news-text').get_text()
    text = text.replace('\n',' ').replace('\xa0', ' ').replace('\r', '').replace('\t', '')
    news_payload = {
    "date_posted" : date_posted,
    "news_link" : nlink,
    "news_text" : text,
    "news_title" : news_title
    }
    print(news_payload)
    collection_name = dbname["focus_scrape_1"]
    collection_name.insert_many([news_payload])
  except:
    pass